# Introduction to _explorationlib_ - Lab

This section has two goals. First to get you familiar with the python library used throughout to simulate and explore exploration. Second, is to get familiar with running that code online using Google's colaboratory.


Google colab is a simple free way to run python code. See the _python_ chapter for more on it. Most of the chapters and assignments in this book have a button to open in colab. This notebook is a simple compact test for all the others. A quick way to make sure the rest should work.

This section has two assignments.

## Step 1

Use the button to open this assignment in a colab. Once it is open, if it is open, run all the cells. Read each cell, then run it, that is. This simple test of the colab is also a good but basic introduction to _explorationlib_, which is the basis for all the experiments and assignments you will see.

If there are no errors, celebrate and consider _this_ assignment complete.

_Note:_ I assume that you, reader, are familiar with python programming already. If you are not, see the _Introduction to python_ assignment.

In [1]:
# (The magic '%' below will put the plot nicely in place)
%matplotlib inline 

# Put your code here.